In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
with open('homework4_spikes.npz', 'rb') as loadfile:
    spike_times = np.load(loadfile, allow_pickle=True)['spike_times']
#     convert array into dataframe
#     DF = pd.DataFrame(spike_times)
#     # save the dataframe as a csv file
#     DF.to_csv("HW4_spike_times.csv")
# print(DF.shape)

with open('homework4_metadata.npy', 'rb') as loadfile:
    metadata = np.load(loadfile)
    time_touch_held = metadata['time_touch_held'] # target onset times for each trial
    time_go_cue = metadata['time_go_cue'] # go cue time for each trial
    time_target_acquired = metadata['time_target_acquired'] # time the target was touched
    trial_reach_target = metadata['trial_reach_target'] # index of reach target for each trial (0 through 7)
    # note that I've "fixed" the reach targets to be 0-7 rather than 1-8
    target_locations = metadata['target_locations'] # x,y location of each target 
    target_angles = metadata['target_angles'] # angle of each target

In [3]:
# Verify that there are in fact two different plan period durations
np.unique(time_go_cue - time_touch_held)

array([ 755, 1005])

In [4]:
def extract_plan_spikes(window_length=None, 
                        start_offset=None):
    # Return a matrix of spike counts in the plan window.
    # - The default plan window is defined as the time between
    #   time_touch_held and time_go_cue for each trial. NOTE:
    #   not all trials have the same duration!!!
    # - If a "window_length" is specified, return the number
    #   of spikes in the window of that duration. Return -1
    #   if the available plan period is too short for the window
    #   (for example, window_length=1000 but the trial plan
    #   period is just 755 ms). 
    # - If a "start_offset" is specified, return the number of
    #   spikes in the window defined as starting "start_offset"
    #   ms after the time_touch_held. (Use either time_go_cue
    #   or the optional window_length to determine window end.)
    #   If start_offset is so large that it extends past the
    #   time_go_cue, return -1.
    
    if start_offset:
        trial_starts = time_touch_held + start_offset
    else:
        trial_starts = time_touch_held
    
    plan_spikes = []   
    for tx, trialSpikes in enumerate(spike_times):
        if window_length:
            trial_end = trial_starts[tx] + window_length
        else:
            trial_end = time_go_cue[tx]
        
        if (trial_end < trial_starts[tx]) or (trial_end > time_go_cue[tx]):
                plan_spikes.append(-np.ones(len(trialSpikes)))
        else:
            plan_spikes.append(
                np.array([np.sum((st > trial_starts[tx]) & 
                        (st < trial_end)) for st in trialSpikes]))
    return np.array(plan_spikes) # will be 1127 x 190 (number of trials by number of neurons)

In [5]:
plan_spikes = extract_plan_spikes(window_length=750)
print(np.array(plan_spikes).shape)
DF = pd.DataFrame(plan_spikes)
DF.to_csv("plan_spikes.csv")

(1127, 190)


In [6]:
# select the 755 ms plan period trials for testing
short_trials = (time_go_cue - time_touch_held) == 755 # boolean array
print("short_trials", np.array(short_trials).shape)
print(short_trials)
print("Dimension of short trails")
training_trials = [] # will be 8 lists of trials
test_trials = [] # all the leftovers
for c in range(8): # reach targets go from 0 to 7
    target_trials = np.argwhere((trial_reach_target==c)).squeeze()
    # randomly select 25 training trials per direction
    random_training_trials = np.random.choice(target_trials, 25, replace=False)
    training_trials.append(random_training_trials)
    remaining_test_trials = np.setdiff1d(target_trials, random_training_trials)
    
    test_trials.extend(remaining_test_trials)

arr1 = np.array(training_trials)
arr2 = np.array(test_trials)
print("Dimension of training trails", arr1.shape)
print(arr1)
print("Dimension of test trails", arr2.shape)
print(arr2)

short_trials (1127,)
[False False False ...  True False False]
Dimension of short trails
Dimension of training trails (8, 25)
[[ 118  504  172  247  865  198  981  804  122   86  560  410  348  720
    46  758  168  276  428  331  293  817  860  337 1113]
 [ 540  177  459  734  277  181  323  532   90  446   10  124   30 1115
    29  801  371   84    9  973  813  288 1109  144  959]
 [ 527  302  853  317   59  151  538  745  421  408  167   39  885  511
    35  854   81  883  519 1073  829  976  694  722  837]
 [  95  646    7  926  693  766  284  759 1007  799   63  770  441  365
   378  673  563  716  841  605  627  272  902  782  888]
 [ 611  847  958  516 1117  474  681  241  975  718  846  164  303  335
   826  211  650  149 1000  219  412  769  366  396  201]
 [ 838  753  498   16  923  579  274  294 1028  619  170  904  859    2
   567  591  557  158  174  295  843  266   25  899   82]
 [ 864  185  435  604  244  691 1008  787  222  363  148 1046  135  715
   652  154  541  966 

In [7]:
# Calculate the mean spike counts per neuron for each target
num_neurons = plan_spikes.shape[1]
print("num_neurons")
print(num_neurons)
mean_spike_counts = np.zeros((num_neurons, 8))
print("mean_spike_counts")
print(mean_spike_counts)
for c in range(8):
    mean_spike_counts[:,c] = np.mean(plan_spikes[training_trials[c],:], axis=0)
    print("training_trials[c] at ", c)
    print(training_trials[c])
    print("plan_spikes[training_trials[c],:] at ", c)
    print(plan_spikes[training_trials[c],:])
    print("mean_spike_counts at ", c)
    print(mean_spike_counts)

num_neurons
190
mean_spike_counts
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
training_trials[c] at  0
[ 118  504  172  247  865  198  981  804  122   86  560  410  348  720
   46  758  168  276  428  331  293  817  860  337 1113]
plan_spikes[training_trials[c],:] at  0
[[ 0  1  3 ...  4 13  5]
 [ 0  3  4 ...  8 17  7]
 [ 0  1  2 ...  8  8  9]
 ...
 [ 0  3  2 ...  3  6  7]
 [ 2  1  1 ... 13  5  6]
 [ 1  5  0 ...  4 31 11]]
mean_spike_counts at  0
[[0.56 0.   0.   ... 0.   0.   0.  ]
 [3.4  0.   0.   ... 0.   0.   0.  ]
 [2.28 0.   0.   ... 0.   0.   0.  ]
 ...
 [6.88 0.   0.   ... 0.   0.   0.  ]
 [8.72 0.   0.   ... 0.   0.   0.  ]
 [5.4  0.   0.   ... 0.   0.   0.  ]]
training_trials[c] at  1
[ 540  177  459  734  277  181  323  532   90  446   10  124   30 1115
   29  801  371   84    9  973  813  288 1109  144  959]
plan_spikes[training_trials[c],:] at  1
[[2 5 3 ... 7 0 

In [8]:
def multivariate_poisson_logpdf(mu, x, mean_eps=0.01): 
    # assume mu is (N,) and x is (d,N)
    mu2 = mu
    mu2[np.argwhere(mu < mean_eps)] = mean_eps
    return np.sum(x * np.log(mu) - mu, axis=1)

In [9]:
# Calculate probability for each target and each test trial
poisson_likelihood = np.zeros((len(test_trials), 8))    
for c in range(8):
    m = mean_spike_counts[:,c]
    poisson_likelihood[:,c] = \
        multivariate_poisson_logpdf(m, plan_spikes[test_trials,:])


In [10]:
correct_targets = trial_reach_target[test_trials] # correct target for each trial
decoded_targets = np.argmax(poisson_likelihood,axis=1) # decoded target is just the max index
print('Poisson Correct: ', np.mean(correct_targets==decoded_targets))
print('({} test trials)'.format(len(test_trials)))

Poisson Correct:  0.9277238403451996
(927 test trials)


In [12]:
for offset in range(0, 500, 50):
    decode_perf = []
    for plan_window in np.arange(50,800,50): # (this goes to 750!!!)
        plan_spikes = extract_plan_spikes(window_length=250, start_offset=offset)  #set the length of the decode window to 250 ms
        # Before: plan_spikes = extract_plan_spikes(window_length=plan_window)
        #  change your loop to loop over an offset variable that you design to range from 0 to 500 (500 is the maximum offset that will fit for the 755 ms plan windows!)
        decode_perf.append([])
    
        # Randomly choose training and test data 25 times
        for i in range(25): 
        
            # Fill in code here to 
            #  1) randomly select training and test trials    

            # randomly select 25 training trials per direction

            training_trials = [] # will be 8 lists of trials
            test_trials = [] # all the leftovers
            for c in range(8): # reach targets go from 0 to 7
                target_trials = np.argwhere((trial_reach_target==c)).squeeze()
                # randomly select 25 training trials per direction
                random_training_trials = np.random.choice(target_trials, 25, replace=False)
                training_trials.append(random_training_trials)
                remaining_test_trials = np.setdiff1d(target_trials, random_training_trials)

                test_trials.extend(remaining_test_trials)
            num_neurons = plan_spikes.shape[1]
            mean_spike_counts = np.zeros((num_neurons, 8))
            for c in range(8):
                #  2) calculate mean spike counts using training data
                mean_spike_counts[:,c] = np.mean(plan_spikes[training_trials[c],:], axis=0)

            poisson_likelihood = np.zeros((len(test_trials), 8))
            for c in range(8):
                m = mean_spike_counts[:,c] 
                #  3) calculate target likelihoods for test data
                poisson_likelihood[:,c] = multivariate_poisson_logpdf(m, plan_spikes[test_trials, :])

            #  4) decode by picking the most likely target
            correct_targets = trial_reach_target[test_trials]
            decoded_targets = np.argmax(poisson_likelihood,axis=1)

            # Assuming that you define the variables "correct_targets"
            #  and "decoded_targets", the follow line calculates
            #  the decoding performance for this iteration

            decode_perf[-1].append(np.mean(correct_targets==decoded_targets))

    decode_perf = np.array(decode_perf) # convert to a numpy array 
    #print(decode_perf)
    
    fig = plt.figure(figsize=(10,10))
    ax = sns.violinplot(data = pd.DataFrame(np.array(decode_perf).T, 
                                  columns=np.arange(50,800,50)))
    ax.set(xlabel='Plan Window Duration', ylabel='Decode Accuracy')
    ax.set_ylim([0.7, 1])

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (25,) (190,) 